In [1]:
from bs4 import BeautifulSoup
import json
import re
import time
import datetime
import os

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#PATH = '../../../chromedriver_linux64/chromedriver'

In [3]:
chrome_options = Options()
chrome_options.add_argument("--headless")

In [4]:
PRICE = r'<span class="st_3OTz8Ec">(\d+\.\d+)</span>'
PRICE_CHANGE = r'<span class="st_19n_1MZ">(\d+\.\d+)</span>'
PRICE_CHANGE_DOLLARS = r'<span class="st_3PJUnZB">(\d+\.\d+)</span><span>'
PRICE_CHANGE_PERCENT = r'-->(\d+\.\d+)%<!--'
#COMMENTS = r'<div class="st_3SL2gug">(.*?)<div class="st_2c27B4Z st_jGV698i st_1GuPg4J st_3QTv-Ni">'
ALL_COMMENTS = r'<article class="st_VgdfbpJ">(.*?)</article>'
COMMENT_SECTION = r'<div class="st_2giLhWN st_1vC-yaI st_yCtClNI st_2mehCkH st_3PPn_WF st_DmhifDD st_2-AYUR9">(.*)<div class="st_2c27B4Z st_jGV698i st_1GuPg4J st_3QTv-Ni">'
BETWEEN_SPAN = '>(\d+\.\d+)<'

In [5]:
def handle_comments(HTML):
    # Regex frames
    ALL_COMMENTS = r'<article class="st_VgdfbpJ st_31oNG-n st_3A22T1e st_vmBJz6-">(.*?)</article>'
    COMMENT_SECTION = r'<div class="st_2giLhWN st_1vC-yaI st_yCtClNI st_2mehCkH st_3PPn_WF st_DmhifDD st_2-AYUR9">(.*)<div class="st_2c27B4Z st_jGV698i st_1GuPg4J st_3QTv-Ni">'
    DATE_TIME = r'<a class="st_28bQfzV st_1E79qOs st_3TuKxmZ st_3Y6ESwY st_GnnuqFp st_1VMMH6S"(.*?)<div class="st_2giLhWN st_1vC-yaI st_yCtClNI st_2mehCkH st_3PPn_WF st_DmhifDD st_2-AYUR9">'
    
    # Empty vars
    COMMENTS_LIST = []
    DATE_STAMPS = []
    SENTIMENT = []
    BULLISH = 0
    BEARISH = 0
    NEUTRAL = 0
    
    # Sentiment analysis
    for section in re.findall(ALL_COMMENTS, str(HTML)):
        if 'Bullish' in section:
            SENTIMENT += [1]
        elif 'Bearish' in section:
            SENTIMENT += [-1]
        else:
            SENTIMENT += [0]
                        
        # Collect comments text
        comment = re.search(COMMENT_SECTION,section).group()
        date = re.search(DATE_TIME,section).group()
        
        
        COMMENTS_LIST += [BeautifulSoup(comment).get_text()]
        DATE_STAMPS += [BeautifulSoup('<a '+date).get_text()]
        
        
    # Parse times
    FORMATTED_TIMES = []
    count = 0
    for tim in DATE_STAMPS:
        if 'PM' in tim or 'AM' in tim:
            break
        elif tim == 'now':
            FORMATTED_TIMES += [datetime.datetime.fromtimestamp(time.time()).strftime("%H:%M:%S.%f")]
        else:
            t = int(tim[0:-1])
            if t < 21:
                FORMATTED_TIMES += [datetime.datetime.fromtimestamp(time.time()-t*60).strftime("%H:%M:%S.%f")]
            else:
                break
        count += 1
    
    for i in SENTIMENT[0:count]:
        if i == 1:
            BULLISH += 1
        elif i == -1:
            BEARISH += 1
        else:
            NEUTRAL += 1
    
    print(f'Bullish: {BULLISH}\nBearish: {BEARISH}\nNeutral: {NEUTRAL}')
    COMMENTS = {FORMATTED_TIMES[i]:{"text":COMMENTS_LIST[i],"sentiment":SENTIMENT[i]} for i in range(len(FORMATTED_TIMES))}
    SENTIMENT = {"bullish":BULLISH,"bearish":BEARISH,"neutral":NEUTRAL}
    
    return(COMMENTS,SENTIMENT)


In [6]:
def handle_json(HTML):
    # Regex frame
    JSON_PARSE = r'"symbol":(.*)}},"isFetching"'
    
    # Scrape for frame
    raw_json = re.search(JSON_PARSE, str(HTML)).group()
    
    # Format text to JSON
    JSON_TXT = raw_json
    JSON_TXT = JSON_TXT.replace('},"isFetching"','')
    BODY = JSON_TXT.split(",")[4:]
    TXT = ",".join(BODY)
    JSON_TXT = '{'+TXT
    
    # JSON loads
    TICKER_DATA = json.loads(JSON_TXT)
    
    return(TICKER_DATA)

In [7]:
def push_json_to_csv(JSON,COMMENTS,SENTIMENT,MKTCAP,RATIO,RELATIVE_RATIO,SYMBOL):
    # SENTIMENT = (BULLS-BEARS)/NEUTRAL
    
    globalMetrics = [
        'open',
        'previousClose',
    ]
    
    constMetrics = [
        'trendingScore',
        'trending', #True/False
        'volume',
        'volumeChange',
        'watchlistCount',
        'low',
        'high',
        'price',
        'change',
        'percentChange',
        #'exchange',
        #'id',
        #'previousCloseDate',
        #'lastUpdated', #MM/DD/YYYY, HH:MM:SS AM/PM EDT
    ]

    
    symbol = SYMBOL
    DATE = str(datetime.datetime.now()).split('.')[0].split(' ')[0]
    TIME = str(datetime.datetime.now()).split('.')[0].split(' ')[1][0:-3]
    
    if os.path.isfile(f'stock/{symbol}.json') != True:
        with open(f'stock/{symbol}.json','w') as f:
            json.dump({symbol:{"date":{DATE:{"open": "", "previousClose": "", TIME:{"relativeRatio":"", "ratio":"", "mktcap":"", "comments": {}, "sentiment": {}, "trendingScore": "", "trending": "", "volume": "", "relativeVolume":"", "volumeChange": "", "watchlistCount": "", "low": "", "high": "", "price": "", "change": "", "percentChange":""}}}}}, f)
        
    with open(f'stock/{symbol}.json','r+') as f:
        DATA = json.load(f)
    
    try:
        TODAY_DATA = DATA[symbol]["date"][DATE].copy()
    except:
        TODAY_FRAME = {DATE:{"open": "", "previousClose": ""}}
        DATA[symbol]["date"].update(TODAY_FRAME)
        TODAY_DATA = DATA[symbol]["date"][DATE].copy()
        
    CURRENT_DATA = {"relativeRatio":"", "ratio":"", "mktcap":"", "comments": {}, "sentiment": {}, "trendingScore": "", "trending": "", "volume": "", "relativeVolume":"", "volumeChange": "", "watchlistCount": "", "low": "", "high": "", "price": "", "change": "", "percentChange":""}
                
    for metr in constMetrics:
        CURRENT_DATA[metr] = JSON[metr]
        
    for metr in globalMetrics:
        TODAY_DATA.update({metr:JSON[metr]})
    
    RELATIVE_VOLUME = get_relative_volume(int(JSON["volume"]))
    CURRENT_DATA.update({"relativeVolume":RELATIVE_VOLUME})
    CURRENT_DATA.update({"ratio":RATIO})
    CURRENT_DATA.update({"relativeRatio":RELATIVE_RATIO})
    CURRENT_DATA.update({"mktcap":MKTCAP})
    CURRENT_DATA.update({"comments":COMMENTS})
    CURRENT_DATA.update({"sentiment":SENTIMENT})
        
    TODAY_DATA.update({TIME:CURRENT_DATA})
    DATA[symbol]["date"][DATE] = TODAY_DATA
    with open(f'stock/{symbol}.json','w') as f:
        json.dump(DATA, f)


In [8]:
from yahoo_fin import stock_info as si
# si.get_day_gainers()
# si.get_day_losers()

def get_mktcap_ratio(symbol):
    MKTCAP_DICT = {'K':1000,'M':1000000,'B':1000000000,'T':1000000000000}
    DATA = si.get_quote_table(symbol, dict_result=True)
    MKTCAP = float(DATA['Market Cap'][:-1])*MKTCAP_DICT[DATA['Market Cap'][-1]]
    VOLUME = float(DATA['Volume'])
    RELATIVE_VOLUME = get_relative_volume(int(VOLUME))
    PRICE  = float(DATA['Quote Price'])
    
    
    # Ratio = 1 / ((MktCap / Price) / volume)
    NUM_SHARES = MKTCAP/PRICE
    RATIO = 1 / (NUM_SHARES/VOLUME)
    RELATIVE_RATIO = 1 / (NUM_SHARES/RELATIVE_VOLUME)
    
    return(MKTCAP,RATIO,RELATIVE_RATIO)


In [9]:
def get_relative_volume(volume):
    DATE = str(datetime.datetime.now()).split('.')[0].split(' ')[0]
    TIME = str(datetime.datetime.now()).split('.')[0].split(' ')[1][0:-3]
    
    HOUR = int(TIME[0:2])
    MIN = int(TIME[3:5])
    
    if HOUR < 9 or HOUR > 20:
        return volume
    elif HOUR < 9 and MIN < 30:
        return volume
    else:
        ELAPSED = (HOUR*3+int(MIN/20))-(9*3+1.5)
        return volume/ELAPSED

In [10]:
t = time.time()
driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
SYMBOL = 'LK'
driver.get(f'https://stocktwits.com/symbol/{SYMBOL}')
Y = 10000
driver.execute_script(f"window.scrollTo(0, {Y})") 
last_height = driver.execute_script("return document.body.scrollHeight")

HTML = driver.page_source.encode("utf-8")
driver.quit()
print(time.time()-t)

[WDM] - Cache is valid for [04/06/2020]
[WDM] - Looking for [chromedriver 80.0.3987.106 linux64] driver in cache 
[WDM] - Driver found in cache [/home/lukepiette/.wdm/drivers/chromedriver/80.0.3987.106/linux64/chromedriver]


KeyboardInterrupt: 

In [15]:
def cycle_data(tim,SYMBOLS=[]):
    Y = 20000
    for i in range(tim):
        driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
        for symb in SYMBOLS:
            print(symb)
            t = time.time()
            driver.get(f'https://stocktwits.com/symbol/{symb}')
            driver.execute_script(f"window.scrollTo(0, {Y})") 
            #last_height = driver.execute_script("return document.body.scrollHeight")

            HTML = driver.page_source.encode("utf-8")
            JSON = handle_json(HTML)
            COMMENTS,SENTIMENT = handle_comments(HTML)
            MKTCAP,RATIO,RELATIVE_RATIO = get_mktcap_ratio(symb)
            push_json_to_csv(JSON,COMMENTS,SENTIMENT,MKTCAP,RATIO,RELATIVE_RATIO,symb)
            
            print(f'{symb} done in {time.time()-t}\n')

        driver.quit()
        print('Sleeping...\n')
        time.sleep(20*60)

In [16]:
SYMBOLS = ['LK','XSPA','PRTY','HTZ']
cycle_data(2,SYMBOLS=SYMBOLS)

[WDM] - Cache is valid for [04/06/2020]
[WDM] - Looking for [chromedriver 80.0.3987.106 linux64] driver in cache 
[WDM] - Driver found in cache [/home/lukepiette/.wdm/drivers/chromedriver/80.0.3987.106/linux64/chromedriver]


 
LK
Bullish: 7
Bearish: 0
Neutral: 13
LK done in 6.62455415725708

XSPA
Bullish: 13
Bearish: 0
Neutral: 7
XSPA done in 5.240666389465332

PRTY
Bullish: 2
Bearish: 0
Neutral: 0
PRTY done in 5.269944906234741

HTZ
Bullish: 4
Bearish: 1
Neutral: 6
HTZ done in 8.15838623046875

Sleeping...



KeyboardInterrupt: 

In [17]:
with open(f'stock/{SYMBOL}.json','r') as f:
    parsed = json.load(f)
    print(json.dumps(parsed, indent=2, sort_keys=False))

{
  "LK": {
    "date": {
      "2020-06-04": {
        "open": 2.7,
        "previousClose": 2.58,
        "12:31": {
          "relativeRatio": 0.056399036617365124,
          "ratio": 0.4793918112476035,
          "mktcap": 1056000000.0,
          "comments": {
            "12:31:33.861866": {
              "text": "$LK remember only in Jan it was a $50 stock, then people panicked due to fraud, everyone gets 2nd chance. SO much potential on the up side. Can makes fortunes for bulls who play it smartly  (Buy & hold)",
              "sentiment": 1
            },
            "12:31:33.861909": {
              "text": "$LK fraudulent financials but I\\xe2\\x80\\x99ll ride a pump",
              "sentiment": 0
            },
            "12:31:33.861920": {
              "text": "$LK Once $4.30 breaks, $20 is in the cards.",
              "sentiment": 1
            },
            "12:31:33.861931": {
              "text": "$LK still here from before the halt , holding the bag still , whe